In [1]:
using Random
using Statistics
using LinearAlgebra
using Plots
using Polynomials
using JLD2
import Tensor_FixedSeedMC as TTMC

In [2]:
#import Pkg; Pkg.add("Polynomials")

In [3]:
# 標準正規乱数を生成する関数
function gen_sn(M, I; anti_paths=true, mo_match=true)
    Random.seed!(1234)
    if anti_paths
        sn = randn(M + 1, Int(I / 2))
        sn = hcat(sn, -sn)
    else
        sn = randn(M + 1, I)
    end
    if mo_match
        sn = (sn .- mean(sn)) ./ std(sn)
    end
    return sn
end

# アメリカン・プットオプションの価格を計算する関数
function price_american_put_option(K::Float64, S0::Float64, r::Float64, sigma::Float64, T::Float64; M::Int=50, I::Int=10000)
    dt = T / M                       # タイムステップの長さ
    df = exp(-r * dt)                # 割引ファクター
    S = zeros(M + 1, I)              # 資産価格パスの行列
    S[1, :] .= S0                    # 初期資産価格を設定
    sn = gen_sn(M, I)                # 標準正規乱数の生成

    # 資産価格パスのシミュレーション
    for t in 2:M+1
        S[t, :] = S[t - 1, :] .* exp.((r - 0.5 * sigma^2) * dt .+ sigma * sqrt(dt) .* sn[t, :])
    end

    h = max.(K .- S, 0.0)            # 即時行使価値
    V = copy(h)                      # オプション価値行列

    # Longstaff-Schwartzアルゴリズムの実装（逆順ループ）
    for t in M:-1:2
        discounted_V = V[t + 1, :] * df   # 次期のオプション価値を現在価値に割引

        # 回帰モデルのフィッティング（次数3の多項式）
        p = fit(S[t, :], discounted_V, 3)

        # 継続価値の推定
        C = p.(S[t, :])

        # 行使戦略の決定
        V[t, :] = ifelse.(C .> h[t, :], V[t + 1, :] * df, h[t, :])
    end

    C0 = df * mean(V[2, :])               # オプション価格の推定
    err = 1.96 * df * std(V[2, :]) / sqrt(I)     # オプション価格の標準誤差
    return C0, err
end

# 固定パラメータ
S0 = 100.0      # 初期資産価格
r = 0.05        # 無リスク金利
T = 1.0         # 満期（年）
M = 365         # タイムステップ数
I = 100000       # シミュレーションパス数

# 変化させるパラメータの範囲
S0_range = collect(90:0.3:119.7) # 200
sigma_range = collect(0.15:0.001:0.249) #200
# オプション価格を格納する行列
option_price = zeros(length(sigma_range), length(S0_range))


K = 100.0
sigma = sigma_range[1]
option_price, err = price_american_put_option(K, S0, r, sigma, T, M=M, I=I)

(4.03366581525055, 0.027903763819380693)

In [4]:
random_combinations = TTMC.generate_random_combinations(length(S0_range), 2, 100);

In [5]:
prices = []
errs = []
for i in random_combinations
    S0 = S0_range[i[1]]
    sigma = sigma_range[i[2]]
    option_price, err = price_american_put_option(K, S0, r, sigma, T, M=M, I=I)
    push!(prices, option_price)
    push!(errs, err)
end

res = [prices, errs]

2-element Vector{Vector{Any}}:
 [7.909418418891762, 4.737036795495581, 9.806194499548939, 2.5395569735834957, 2.4373539463045173, 2.3778287454208615, 5.533384346529686, 7.481621164946778, 0.8769836700527268, 2.242949061386431  …  1.6664228175341282, 5.771906155026692, 6.309611931843609, 5.119210837773977, 2.0183682322201766, 3.0136638382410132, 6.626969678864436, 5.7654179434962485, 1.6511338538410008, 8.627965013584236]
 [0.036576763448995415, 0.0419489971254994, 0.03882002474019096, 0.03155559375527327, 0.023490149816698995, 0.02810061302243763, 0.03599698856130929, 0.04319011427040661, 0.015282099133788389, 0.026281331340740264  …  0.022838656903049044, 0.04214477371933169, 0.04019430434231982, 0.04378785625955837, 0.02666879038831784, 0.03165946875134441, 0.04745900583031191, 0.03642187237890039, 0.02446303554595037, 0.049466676602255816]

In [6]:
JLD2.@save "american_S0_sigma_true.jld2" res

In [2]:
using JLD2
JLD2.@load "american_S0_sigma_true.jld2" res
res = res

2-element Vector{Vector{Any}}:
 [7.909418418891762, 4.737036795495581, 9.806194499548939, 2.5395569735834957, 2.4373539463045173, 2.3778287454208615, 5.533384346529686, 7.481621164946778, 0.8769836700527268, 2.242949061386431  …  1.6664228175341282, 5.771906155026692, 6.309611931843609, 5.119210837773977, 2.0183682322201766, 3.0136638382410132, 6.626969678864436, 5.7654179434962485, 1.6511338538410008, 8.627965013584236]
 [0.036576763448995415, 0.0419489971254994, 0.03882002474019096, 0.03155559375527327, 0.023490149816698995, 0.02810061302243763, 0.03599698856130929, 0.04319011427040661, 0.015282099133788389, 0.026281331340740264  …  0.022838656903049044, 0.04214477371933169, 0.04019430434231982, 0.04378785625955837, 0.02666879038831784, 0.03165946875134441, 0.04745900583031191, 0.03642187237890039, 0.02446303554595037, 0.049466676602255816]